In [5]:
from modeling import get_training_model
from run_training_encoders import __model_file_pattern__, __feature_data__, __label__, split_train_test_valid
import pandas as pd
import numpy as np
import os
kfold = pd.read_csv("Dataset_10FoldCV_indexed.csv")
i = 0
for feature in __feature_data__.keys():
    model = get_training_model(feature)
    X = np.load(__feature_data__[feature])
    y = np.load(__label__)
    
    for l2_rate in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]:
        valid_accuracies, test_accuracies = [],[]
        for fold in range(1,11):
            (X_train, y_train), (X_valid, y_valid), (X_test, y_test) = split_train_test_valid(feature, kfold, fold, X, y)
            model_path = __model_file_pattern__.format(feature,l2_rate, 0.5, fold)
            model_path = os.path.join("Dataset_10FoldCV_indexed_models/", model_path)
            model.load_weights(model_path)
            _, val_acc = model.evaluate(X_valid, y_valid, verbose=0)
            _, test_acc = model.evaluate(X_test, y_test, verbose=0)   
            valid_accuracies.append(val_acc)
            test_accuracies.append(test_acc)
            
            print("Feature {} - fold {} - val_acc - {} - test_acc {}".format(feature, fold, val_acc, test_acc))
            performances.loc[i] = ["Dataset_10FoldCV_indexed.csv", fold, feature, l2_rate, 0.5, val_acc, test_acc]
            i += 1

        print("Valid accuracy: {} +- {}".format(np.mean(valid_accuracies), np.std(valid_accuracies)))
        print("Test accuracy: {} +- {}".format(np.mean(test_accuracies), np.std(test_accuracies)))        

Feature image - fold 1 - val_acc - 0.9736841917037964 - test_acc 0.9685863852500916
Feature image - fold 2 - val_acc - 1.0 - test_acc 0.9790576100349426
Feature image - fold 3 - val_acc - 0.9738219976425171 - test_acc 0.9528796076774597
Feature image - fold 4 - val_acc - 0.9895287752151489 - test_acc 0.9947643876075745
Feature image - fold 5 - val_acc - 1.0 - test_acc 0.9790576100349426
Feature image - fold 6 - val_acc - 0.9895287752151489 - test_acc 0.9685863852500916
Feature image - fold 7 - val_acc - 0.9738219976425171 - test_acc 0.9842932224273682
Feature image - fold 8 - val_acc - 0.9947643876075745 - test_acc 0.9736841917037964
Feature image - fold 9 - val_acc - 0.9789473414421082 - test_acc 0.9736841917037964
Feature image - fold 10 - val_acc - 1.0 - test_acc 0.9684210419654846
Valid accuracy: 0.987409770488739 +- 0.010806811042129993
Test accuracy: 0.9743014574050903 +- 0.010598914697766304
Feature image - fold 1 - val_acc - 0.9684210419654846 - test_acc 0.9790576100349426
Feat

In [7]:
performances.to_csv("encoders_performances.csv", index=False)

In [52]:
performances = pd.DataFrame(columns=performances.columns)

In [59]:
best_encoders = pd.DataFrame(columns=["kfold_file", "fold", "feature", "l2_rate", "dropout_rate", "val_acc", "test_acc"])
#best_encoders = best_encoders.reindex(columns=["kfold_file", "fold", "feature", "l2_rate", "dropout_rate", "val_acc", "test_acc"])

In [60]:
i = 0
for feature in __feature_data__.keys():
    best_val_acc = 0.0
    for l2_rate in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]:
        architecture_perform = performances[np.logical_and(performances['feature'] == feature, performances['l2_rate'] == l2_rate)]
        val_acc = np.mean(architecture_perform['val_acc'])
        if val_acc >= best_val_acc:
            best_val_acc = val_acc
            best_l2_rate = l2_rate
    best_encoders = pd.concat([best_encoders, performances[np.logical_and(performances['feature'] == feature, performances['l2_rate'] == best_l2_rate)]], sort=False)


In [63]:
best_encoders.to_csv("best_encoders.csv", index=False)

In [32]:
## best encoder each fold each feature

from modeling import get_training_model
from run_training_encoders import __model_file_pattern__, __feature_data__, __label__, split_train_test_valid
import pandas as pd
import numpy as np
import os
performances = pd.read_csv("encoders_performances.csv")
best_encoders = pd.DataFrame(columns=performances.columns)
i = 0
for fold in range(1,11):
    for feature in __feature_data__.keys():
        encoders = performances[np.logical_and(performances['feature'] == feature, performances['fold'] == fold)]
        best_encoder = encoders.loc[encoders['val_acc'].idxmax()]
        best_encoders.loc[i] = best_encoder
        i += 1

In [33]:
best_encoders

,kfold_file,fold,feature,l2_rate,dropout_rate,val_acc,test_acc
0,Dataset_10FoldCV_indexed.csv,1,image,0.00100,0.5,0.984211,0.994764
1,Dataset_10FoldCV_indexed.csv,1,vein,0.00001,0.5,0.968421,0.952880
2,Dataset_10FoldCV_indexed.csv,1,xyprojection,0.01000,0.5,0.831579,0.816754
3,Dataset_10FoldCV_indexed.csv,1,color,0.01000,0.5,0.884211,0.884817
4,Dataset_10FoldCV_indexed.csv,1,texture,0.00001,0.5,0.815789,0.780105
5,Dataset_10FoldCV_indexed.csv,1,fourier,0.10000,0.5,0.815789,0.759162
6,Dataset_10FoldCV_indexed.csv,1,shape,0.00100,0.5,0.900000,0.900524
7,Dataset_10FoldCV_indexed.csv,2,image,0.00001,0.5,1.000000,0.979058
8,Dataset_10FoldCV_indexed.csv,2,vein,0.00001,0.5,0.989529,0.973822
9,Dataset_10FoldCV_indexed.csv,2,xyprojection,1.00000,0.5,0.842932,0.832461


In [51]:
old_best_encoders = pd.read_csv("best_encoders.csv")

In [52]:
for feature in __feature_data__.keys():
    best_encoder = best_encoders[best_encoders['feature'] == feature]
    print(feature)
    print("Valid: {} +- {}".format(np.mean(best_encoder['val_acc']), np.std(best_encoder['val_acc'])))
    print("Test: {} +- {}".format(np.mean(best_encoder['test_acc']), np.std(best_encoder['test_acc'])))
    print()
    best_encoder = old_best_encoders[old_best_encoders['feature'] == feature]
    print(feature)
    print("Valid: {} +- {}".format(np.mean(best_encoder['val_acc']), np.std(best_encoder['val_acc'])))
    print("Test: {} +- {}".format(np.mean(best_encoder['test_acc']), np.std(best_encoder['test_acc'])))
    print()
    #val_acc, test_acc = best_encoder['val_acc'], best_encoder['test_acc']


image
Valid: 0.9947533726692199 +- 0.004701378151008385
Test: 0.9826839327812195 +- 0.007827513008867821

image
Valid: 0.9916037380695343 +- 0.006314456046819216
Test: 0.9842628836631775 +- 0.008462783956728403

vein
Valid: 0.9805924534797669 +- 0.008150912314858856
Test: 0.9554202258586884 +- 0.015776463943011475

vein
Valid: 0.9732626080513 +- 0.012914591612382436
Test: 0.9580380320549011 +- 0.012439083286002479

xyprojection
Valid: 0.859484714269638 +- 0.022384966513109263
Test: 0.8248388051986695 +- 0.01878108691282577

xyprojection
Valid: 0.850584191083908 +- 0.026510721493729413
Test: 0.8175117075443268 +- 0.022456808160265924

color
Valid: 0.8998181283473968 +- 0.021413589030247322
Test: 0.8877845168113708 +- 0.01659062860869922

color
Valid: 0.8909065842628479 +- 0.023649228501934384
Test: 0.8872637093067169 +- 0.019496098659993282

texture
Valid: 0.8458115100860596 +- 0.023486716113641833
Test: 0.8332598567008972 +- 0.02694453239008852

texture
Valid: 0.8395150125026702 +- 0.0

0.9947533726692199

In [45]:
np.mean(best_encoder['test_acc'])

0.9826839327812195